In [43]:
#Importing sim4life modules
import s4l_v1.simulation.neuron as neuron
import s4l_v1.document as document
import s4l_v1.model as model
from s4l_v1.model import Vec3, Translation, Transform, Rotation, Scaling
import random
import numpy as np
import math
import s4l_v1.simulation.emlf as emlf
import XCoreModeling
import XCore
import s4l_v1.materials.database as database

def fascicle_creator(nerve_length,f_diameter,centre_location,axon_no, fascicle_no, a_dia, weights):
    axon_list = []
    
    peri_width = (0.03*f_diameter)+f_diameter/2 
    
    diameter_grouping = [ [] for _ in range(len(a_dia)) ]
    
    overlap_x = []
    overlap_y = []
    
    for n in range(fascicle_no):
        next_fascicle = False
        print('\n',n+1, 'fascicle')
        while next_fascicle == False:
            next_fascicle=True
            distance = np.random.uniform(0, nerve_radius-(2*nerve_radius*0.05))
            theta = np.random.uniform(0, 2*math.pi)
            x=centre_location[0]+distance*math.cos(theta)
            y=centre_location[1]+distance*math.sin(theta)
            overlap_x.append(x)
            overlap_y.append(y)
        
            for t in range(len(overlap_x)):

                if n > 0 and t > 0:
                    if ((-0.35+overlap_x[t-1] < x < 0.35+overlap_x[t-1]) and (-0.35+overlap_y[t-1] < y < 0.35+overlap_y[t-1]))==True:
                        #print(t, 'oops')
                        del overlap_x[-1]
                        del overlap_y[-1]
                        next_fascicle=False
                        break
                    else:
                        next_fascicle=True
                else:
                    next_fascicle=True
                        
        
        Fascicle = model.CreateSolidCylinder( Vec3(x,y,0), Vec3(x,y,nerve_length), f_diameter/2)
        Fascicle.Name = 'Endoneurium_' + str(n+1)
        Fascicle.Transparency = 0.6 
        
        Perineurium = model.CreateSolidTube( Vec3(x,y,0), Vec3(0,0,nerve_length), peri_width, f_diameter/2)
        Perineurium.Name = 'Perineurium_' + str(n+1)
        Perineurium.Transparency = 0.6
    
        EndoGroup.Add(Fascicle)
        PeriGroup.Add(Perineurium)
        
        dia = random.choices(a_dia, weights=weights, k=axon_no)
        #print(dia)
        
        for m in range(axon_no):
            distance = np.random.uniform(0, (f_diameter/2)-(f_diameter*0.1))
            theta = np.random.uniform(0, 2*math.pi)
            
            a_x=x+(distance*math.cos(theta))
            a_y=y+(distance*math.sin(theta))

            Axon = model.CreatePolyLine([Vec3(a_x,a_y,0), Vec3(a_x,a_y,nerve_length)])
            Axon.Name = 'Fascicle_' + str(n+1) + '_Axon_' + str(m+1)
            AxonGroup.Add(Axon)
            
            model_properties = model.SennNeuronProperties()
            model_properties.AxonDiameter = dia[m]
        
            discretized_axon = model.CreateAxonNeuron(Axon, model_properties)
            axon_list.append(discretized_axon)

            for z in range(len(a_dia)):
                if a_dia[z] == dia[m]:
                    diameter_grouping[z].append(discretized_axon)
        
    for a in range(len(a_dia)):
        SubNerveGroup = model.CreateGroup(str(a_dia[a])+"um")
        SubNerveGroup.Add(diameter_grouping[a])
        AxonGroup.Add(SubNerveGroup)
    
    return axon_list

def encapsulation_tissue(nerve_radius, centre_location, cuff_length, cuff_position, cuff_name, growth_stage): #between 0 and 1
    
        tissue_width = nerve_radius+(nerve_radius*1.1)*growth_stage
        Encap = model.CreateSolidTube( Vec3(centre_location[0],centre_location[1],cuff_position), Vec3(0,0,cuff_length), tissue_width, nerve_radius)
        Encap.Name = cuff_name + '_Encap_Tissue'   
        TissueGroup.Add(Encap)
        
        return tissue_width

def ref_epineurium():
    
    entities = []
    entity_names=list(n.Name for n in list(model.AllEntities())) # from Leen Jabban
    print(entity_names)
    entity_names.remove("Model")
    entity_names.remove("Grid")
    entity_names.remove("Epineurium_Original")
    remove_groups(entity_names)
    
    for i in range(len(entity_names)):
        entities.append(model.AllEntities()[entity_names[i]])
    coord_1, coord_2 = XCoreModeling.GetBoundingBox(entities)
    
    x = (coord_2[0]+coord_1[0])/2
    y = (coord_2[1]+coord_1[1])/2
    print(coord_1, coord_2)
    
    length = round(coord_2[2]-coord_1[2], 5)
    size_Y = round(coord_2[1]-coord_1[1], 5)
    size_X = round(coord_2[0]-coord_1[0], 5)
    print(length,size_X,size_Y)
    
    #XCoreModeling.CreatePoint(Vec3(x,y,length))
    centre = [x,y]
    #diagonals = [XCoreModeling.CreatePoint(Vec3(coord_2))]
    radius = math.dist([x,y],[coord_1[0],coord_1[1]])

    Epineurium = model.CreateSolidCylinder( Vec3(x,y,0), Vec3(x,y,length), radius)
    Epineurium.Name = 'Epineurium'
    Epineurium.Transparency = 0.6
    TissueGroup.Add(Epineurium) ###########
    
    return length, radius, centre
    
def ref_fascicle_populator(axon_no, a_dia, weights): 
    axon_list = []
    
    # a_dia = [4,10]
    # weights = [0.9,1]
    diameter_grouping = [ [] for _ in range(len(a_dia)) ]
    
    fascicles = 0
    count = -1
    for entity in model.AllEntities():
        count+=1
        if entity.Name.startswith('Endoneurium_'): 
            fascicles+=1

            endo = entity.Clone()
            peri = entity.Clone()
            entity.Delete()
            endo.Name = "Endoneurium_" + str(fascicles)
            peri.Name = "Perineurium_" + str(fascicles)
            coord_1, coord_2 = XCoreModeling.GetBoundingBox([peri])
        
            x = (coord_2[0]+coord_1[0])/2
            y = (coord_2[1]+coord_1[1])/2
            #print(coord_1, coord_2)
    
            length = round(coord_2[2]-coord_1[2], 5)
            size_Y = round(coord_2[1]-coord_1[1], 5)
            size_X = round(coord_2[0]-coord_1[0], 5)
            #print(length,size_X,size_Y)
    
            point = XCoreModeling.CreatePoint(Vec3(x,y,length))
            point.Name = 'Point ' + str(fascicles)
            centre = [x,y]
            #diagonals = [XCoreModeling.CreatePoint(Vec3(coord_2))]
            radius = math.dist([x,y],[coord_1[0],coord_1[1]])
            peri.ApplyTransform(Scaling(Vec3(1.03,1.03,1),Vec3(x,y,0)))
            PeriGroup.Add(peri)
            EndoGroup.Add(endo)
        
            dia = random.choices(a_dia, weights=weights, k=axon_no)
            #print(dia)
        
            for m in range(axon_no):
                distance = np.random.uniform(0, (radius/2)-((radius/2)*0.2))
                theta = np.random.uniform(0, 2*math.pi)
            
                a_x=x+(distance*math.cos(theta))
                a_y=y+(distance*math.sin(theta))

                Axon = model.CreatePolyLine([Vec3(a_x,a_y,0), Vec3(a_x,a_y,length)])
                Axon.Name = 'Fascicle_' + str(fascicles) + '_Axon_' + str(m+1)
                AxonGroup.Add(Axon)
            
                model_properties = model.SennNeuronProperties()
                model_properties.AxonDiameter = dia[m]
        
                discretized_axon = model.CreateAxonNeuron(Axon, model_properties)
                axon_list.append(discretized_axon)

                for z in range(len(a_dia)):
                    if a_dia[z] == dia[m]:
                        diameter_grouping[z].append(discretized_axon)
        
    for a in range(len(a_dia)):
        SubNerveGroup = model.CreateGroup(str(a_dia[a])+"um")
        SubNerveGroup.Add(diameter_grouping[a])
        AxonGroup.Add(SubNerveGroup)
    
    return axon_list

def remove_groups(entity_list):
    entity_list.remove("Tissue")
    entity_list.remove("Perineurium")
    entity_list.remove("Endoneurium")
    entity_list.remove("Axon Trajectories")
    entity_list.remove("Electrodes")
    entity_list.remove("Cuffs")
    entity_list.remove("Contacts")
    entity_list.remove('Boundary Conditions')

def create_cuff(nerve_radius, cuff_position, cuff_name, centre_location, cuff_length, electrode_no, sensor):
    cuff_list = []
    
    Cuff = model.CreateSolidTube( Vec3(centre_location[0],centre_location[1],cuff_position), Vec3(0,0,cuff_length), nerve_radius+0.5, nerve_radius)
    Cuff.Name = cuff_name + '_Cuff'
    cuff_list.append(Cuff.Name)
    Cuff.Transparency = 0.6     
    CuffGroup.Add(Cuff)
    
    BC_list = create_ringelectrode(electrode_no, cuff_position, cuff_length, centre_location, nerve_radius, cuff_name)
    
    if sensor==True:
        create_flux_sensor(cuff_position, cuff_length, nerve_radius, centre_location)
    
    return BC_list

def create_flux_sensor(cuff_position, cuff_length, nerve_radius, centre_location):
    
    flux_sensor = model.CreateSolidBlock(Vec3(-nerve_radius-1+centre_location[0],-nerve_radius-1+centre_location[1],cuff_position+(cuff_length+2)), Vec3(nerve_radius+1+centre_location[0],nerve_radius+1+centre_location[1],cuff_position-2))
    flux_sensor.Name = 'Flux_sensor_object'
    flux_sensor.Transparency = 0.4
    
def create_ringelectrode(electrode_no, cuff_position, cuff_length, centre_location, nerve_radius, cuff_name):
    electrode_list = []
    BC_list = []
    
    electrode_position = (cuff_length/electrode_no)/2
    
    for i in range(electrode_no):
        Contact = model.CreateSolidTube( Vec3(centre_location[0],centre_location[1],cuff_position+electrode_position/2+electrode_position*(2*i)), Vec3(0,0,electrode_position), nerve_radius+0.2, nerve_radius+0.1)
        BC = model.CreateSolidTube( Vec3(centre_location[0],centre_location[1],cuff_position+electrode_position/2+electrode_position*(2*i)), Vec3(0,0,electrode_position), nerve_radius+0.2, nerve_radius+0.2)
        
        Contact.Name = cuff_name + '_Contact_' + str(i+1)
        BC.Name = cuff_name + '_BC_' + str(i+1)
        BC_list.append(BC.Name)
        BCGroup.Add(BC)
        ContactsGroup.Add(Contact)
    
    return BC_list

def create_arraycuff(nerve_radius, cuff_position, cuff_name, centre_location, cuff_length, electrode_no, nodes, sensor):
    cuff_list = []
    
    Cuff = model.CreateSolidTube( Vec3(centre_location[0],centre_location[1],cuff_position), Vec3(0,0,cuff_length), nerve_radius+0.5, nerve_radius)
    Cuff.Name = cuff_name + '_Cuff'
    cuff_list.append(Cuff.Name)
    Cuff.Transparency = 0.6     
    CuffGroup.Add(Cuff)
    
    BC_list = create_arrayelectrode(electrode_no, nodes, cuff_position, cuff_length, centre_location, nerve_radius, cuff_name)
    
    if sensor==True:
        create_flux_sensor(cuff_position, cuff_length, nerve_radius, centre_location)
    
    return BC_list

def create_arrayelectrode(circum_electrode_no, electrode_nodes, cuff_position, cuff_length, centre_location, nerve_radius,cuff_name):
    electrode_list = []
    BC_list = []
    
    width =  0.1
    electrode_size = (cuff_length/circum_electrode_no)/electrode_nodes
    if circum_electrode_no < 4:
        electrode_size = (cuff_length/4)/4
    electrode_position = (cuff_length/electrode_nodes)/2
    for j in range(electrode_nodes):
        for i in range(circum_electrode_no):
            Contact = model.CreateSolidBlock( Vec3(centre_location[0]-electrode_size*1.5,centre_location[1]-electrode_size/3,cuff_position+electrode_position+electrode_position*(2*j)), Vec3(centre_location[0]+electrode_size*1.5,centre_location[1]+electrode_size/3,cuff_position+electrode_position+width+electrode_position*(2*j)))
            BC = model.CreateSolidBlock( Vec3(centre_location[0]-electrode_size*1.5,centre_location[1]-electrode_size/3,cuff_position+electrode_position+width+electrode_position*(2*j)), Vec3(centre_location[0]+electrode_size*1.5,centre_location[1]+electrode_size/3,cuff_position+electrode_position+width+electrode_position*(2*j)))
             
            t = Translation(Vec3(0,0,nerve_radius+0.01))
            r = Rotation(Vec3(0,1,0), Vec3(centre_location[0],centre_location[1],cuff_position+electrode_position+electrode_position*(2*j)), np.pi/2)
            new_r = Rotation(Vec3(0,0,1), Vec3(centre_location[0],centre_location[1],cuff_position+electrode_position+electrode_position*(2*j)), (2*np.pi/circum_electrode_no)*i)
            
            Contact.ApplyTransform(t)
            Contact.ApplyTransform(r)
            Contact.ApplyTransform(new_r)
            BC.ApplyTransform(t)
            BC.ApplyTransform(r)
            BC.ApplyTransform(new_r)
            
            Contact.Name = cuff_name + '_Contact_' + str(i+1) + "_Ring_" + str(j+1)
            BC.Name = cuff_name + '_BC_' + str(i+1) + "_Ring_" + str(j+1)
            BC_list.append(BC.Name)
            BCGroup.Add(BC)
            ContactsGroup.Add(Contact)
    
    return BC_list

def create_EM_sim(freq, pos_or_neg, BC_list, sim_name):
    stim = emlf.ElectroQsOhmicSimulation()
    stim.Name = sim_name
    stim.SetupSettings.Frequency = freq
    
    #Boundary Settings
    cathode_BC = stim.AddBoundarySettings()
    cathode_BC.DirichletValue = 0.5
    cathode_BC.Name = "Cathode"

    anode_BC = stim.AddBoundarySettings()
    anode_BC.DirichletValue = -0.5
    anode_BC.Name = "Anode"
    
    zero_BC = stim.AddBoundarySettings()
    zero_BC.DirichletValue = 0
    zero_BC.Name = "Zero Reference"
    
    for x in range(len(pos_or_neg)):
        if pos_or_neg[x]==1:
            stim.Add(cathode_BC, model.AllEntities()[BC_list[x]])
        elif pos_or_neg[x]== -1:
            stim.Add(anode_BC, model.AllEntities()[BC_list[x]])
        elif pos_or_neg[x]== 0:
            stim.Add(zero_BC, model.AllEntities()[BC_list[x]])
                
    # Editing BoundarySettings "Boundary Settings"
    boundary_settings = stim.AllSettings["Boundary Settings"]
    boundary_settings.BoundaryType = boundary_settings.BoundaryType.enum.Flux
    
    ######################################
    #Grid Settings#
    #######################################
    grid_settings = stim.AddManualGridSettings()
    grid_settings.Name = "Nerve & Cuffs"
    grid_settings.MaxStep = [1, 1, 1]
    grid_settings.Resolution = [0.5, 0.5, 1]
    grid_settings.Priority = 0
    stim.GlobalGridSettings.ManualDiscretization = True
    stim.GlobalGridSettings.MaxStep = [1,1, 1]
    stim.GlobalGridSettings.Resolution = [0.05, 0.05, 1]
    stim.GlobalGridSettings.ManualPadding = True
    stim.GlobalGridSettings.BottomPadding = [2, 2, 2]
    stim.GlobalGridSettings.TopPadding = [2, 2, 2]
    BC_grid_settings = stim.AddManualGridSettings()
    BC_grid_settings.Name = "BC"
    BC_grid_settings.MaxStep = [1, 1, 1]
    BC_grid_settings.Resolution = [0.5, 0.5, 1]
    BC_grid_settings.Priority = 1
    
    endo_grid_settings = stim.AddManualGridSettings()
    endo_grid_settings.Name = "Endoneurium"
    endo_grid_settings.MaxStep = [1, 1, 1]
    endo_grid_settings.Resolution = [0.05, 0.05, 1]
    endo_grid_settings.Priority = 0
    
    peri_grid_settings = stim.AddManualGridSettings()
    peri_grid_settings.Name = "Perineurium"
    peri_grid_settings.MaxStep = [1, 1, 1]
    peri_grid_settings.Resolution = [0.01, 0.01, 1]
    peri_grid_settings.Priority = 100
    
    #################################
    #Material & Voxeller Settings#
    ##################################
    cuff_settings = emlf.MaterialSettings()
    stim.LinkMaterialWithDatabase(cuff_settings, database["Generic 1.1"]["Polyimide"])
    cuff_settings.Name='Polyimide'
    cuff_voxeler_settings = emlf.ManualVoxelerSettings()
    cuff_voxeler_settings.Name = "Cuffs"
    cuff_voxeler_settings.Priority = 0
    
    contact_settings = emlf.MaterialSettings()
    stim.LinkMaterialWithDatabase(contact_settings, database["Generic 1.1"]["Platinium"])
    contact_settings.Name='Platinum'
    contact_voxeler_settings = emlf.ManualVoxelerSettings()
    contact_voxeler_settings.Name = "Contacts"
    contact_voxeler_settings.Priority = 1
    
    BC_voxeler_settings = emlf.ManualVoxelerSettings()
    BC_voxeler_settings.Name = "Boundary"
    BC_voxeler_settings.Priority = 2
    
    epi_settings = emlf.MaterialSettings()
    epi_settings.Name="Epineurium"
    epi_settings.ElectricProps.Conductivity = 1 ############################
    epi_voxeler_settings = emlf.ManualVoxelerSettings()
    epi_voxeler_settings.Name = "Epineurium"
    epi_voxeler_settings.Priority = 3
    
    encap_settings = emlf.MaterialSettings()
    encap_settings.Name="Encapsulation Tissue"
    encap_settings.ElectricProps.Conductivity = 6.59e-2 ############################
    encap_voxeler_settings = emlf.ManualVoxelerSettings()
    encap_voxeler_settings.Name = "Encapsulation Tissue"
    encap_voxeler_settings.Priority = 3
    
    endo_settings = emlf.MaterialSettings()
    endo_settings.Name="Endoneurium"
    endo_settings.ElectricProps.ConductivityAnisotropic = True
    endo_settings.ElectricProps.ConductivityDiagonalElements = [1,1,1] ############################
    endo_settings.ElectricProps.ConductivityOffDiagonalElements = [0.1, 0.1, 0.1] ############################
    endo_voxeler_settings = emlf.ManualVoxelerSettings()
    endo_voxeler_settings.Name = "Endoneurium"
    endo_voxeler_settings.Priority = 4 
    
    peri_settings = emlf.MaterialSettings()
    peri_settings.Name="Perineurium"
    peri_settings.ElectricProps.Conductivity = 8.3e-4 ###########################
    peri_voxeler_settings = emlf.ManualVoxelerSettings()
    peri_voxeler_settings.Name = "Perineurium"
    peri_voxeler_settings.Priority = 5 
    
    entity_names=list(n.Name for n in list(model.AllEntities())) # from Leen Jabban
    
    for i in range(len(entity_names)):
        entity = model.AllEntities()[entity_names[i]]
    #count = -1
    #for entity in model.AllEntities():
     #   count+=1
        if entity.Name == sim_name + '_Encap_Tissue': 
            stim.Add(encap_settings, entity)
            stim.Add(encap_voxeler_settings, entity)
            stim.Add(grid_settings, entity)
            
        elif entity.Name == 'Epineurium':
            stim.Add(epi_settings, entity)
            stim.Add(epi_voxeler_settings, entity)
            stim.Add(grid_settings, entity)
            
        elif entity.Name.startswith(sim_name + '_Contact_'):
            stim.Add(contact_settings, entity)
            stim.Add(contact_voxeler_settings, entity)
            stim.Add(grid_settings, entity)
        
        elif entity.Name == sim_name + '_Cuff':
            stim.Add(cuff_settings, entity)
            stim.Add(cuff_voxeler_settings, entity)
            stim.Add(grid_settings, entity)
            
        elif entity.Name.startswith('Perineurium_'):
            stim.Add(peri_settings, entity)
            stim.Add(peri_voxeler_settings, entity)
            stim.Add(peri_grid_settings, entity)
            
        elif entity.Name.startswith('Endoneurium_'):
            stim.Add(endo_settings, entity)
            stim.Add(endo_voxeler_settings, entity)
            stim.Add(endo_grid_settings, entity)
            
        elif entity.Name.startswith(sim_name + '_BC_'):
            stim.Add(BC_voxeler_settings, entity)
            stim.Add(BC_grid_settings, entity)
    
    #Removing AutomaticGridSettings Automatic
    stim.RemoveSettings(stim.AllSettings["Automatic"])
    #Removing AutomaticVoxelerSettings Automatic Voxeler Settings
    stim.RemoveSettings(stim.AllSettings["Automatic Voxeler Settings"])

    # Editing SolverSettings "Solver"
    solver_settings = stim.SolverSettings
    solver_settings.PredefinedTolerances = solver_settings.PredefinedTolerances.enum.High

    # Update the materials with the new frequency parameters
    stim.UpdateAllMaterials()

    # Update the grid with the new parameters and create voxels
    stim.UpdateGrid()
    document.AllSimulations.Add( stim )

def run_EM(sim_name):
    stim = document.AllSimulations[sim_name]
    stim.CreateVoxels()
    stim.RunSimulation() 
    
def run_NEURON(sim_name):
    neuro = document.AllSimulations[sim_name]
    neuro.RunSimulation()
    
def create_NEURON_sim(duration, timestep, axon_list):
    neuro = neuron.Simulation()

    setup_settings = neuro.SetupSettings
    setup_settings.PerformTitration = True
    setup_settings.DepolarizationDetection = setup_settings.DepolarizationDetection.enum.Threshold
    setup_settings.DepolarizationThreshold = 80

    # Neuron Settings
    # neuron_settings = neuron.AutomaticAxonNeuronSettings()
    # for i in range(len(axon_list)):
    #     neuro.Add(neuron_settings, axon_list[i])

    #Link Neuron to Stimulation simulation
    link = document.AllSimulations["Stimulation"]
    nsource= neuro.AddSource(link, "Overall Field")
    nsource.PulseType = nsource.PulseType.enum.Bipolar
    nsource.PulseType = 1
    nsource.Interval = 0.1

    linesensor = neuro.AddOverallLineSensor()
    linesensor.RecordScaledIMembrane = True
    linesensor.RecordV = True
    linesensor.NumberOfSnapshots = 200
    
    # Editing SolverSettings "Solver"
    solver_settings = neuro.SolverSettings
    solver_settings.Duration = duration
    solver_settings.TimeStep= timestep
    solver_settings.NumberOfThreads = 16
  
    document.AllSimulations.Add( neuro )
    
def selectivity_sensor (fascicle_no,nerve_length):
    neuro = document.AllSimulations['Nr']
    
    entity_names=list(n.Name for n in list(model.AllEntities())) # from Leen Jabban
    
    count = 0
    for a in range(fascicle_no):
        print('fascicle',a)
        for i in range(len(entity_names)):
            entity = model.AllEntities()[entity_names[i]]
            if (entity.Name.startswith("Fascicle_" + str(a+1) + "_")) and (entity.Name.endswith(']')):
                axon = entity.Name
        count += 1
        point_sensor = neuro.AddPointSensor(model.AllEntities()[axon])
        point_sensor.Name = point_sensor.Name + str(count)  
        axon_nodes = (len(neuro.GetSectionNames(model.AllEntities()[axon]))/2)/2
        print(axon_nodes)
        point_sensor.SectionName = "node[" + str(round(axon_nodes)) + "]"
        point_sensor.RecordV= True
        point_sensor.RelativePosition = 0.5


In [31]:
#Creating multitree model groups
TissueGroup = model.CreateGroup('Tissue')
PeriGroup = model.CreateGroup('Perineurium')
EndoGroup = model.CreateGroup('Endoneurium')
TissueGroup.Add([PeriGroup, EndoGroup])
AxonGroup = model.CreateGroup('Axon Trajectories')

ElectrodeGroup = model.CreateGroup('Electrodes')
CuffGroup = model.CreateGroup('Cuffs')
ContactsGroup = model.CreateGroup('Contacts')
BCGroup = model.CreateGroup('Boundary Conditions')
ElectrodeGroup.Add([CuffGroup, ContactsGroup, BCGroup])
                
nerve_length = 300
#nerve_radius = 5
centre_position = [0.0,0.0]
cuff_position = 200
cuff_length = 20
#f_diameter = 0.3
#fascicle_no = 30
axon_no=10
electrode_no = 9
pos_or_neg=[1,-1]
cuff_list = [0,0]

#Create Epineurium/Vagus cylinder
# SingleFascicle = model.CreateSolidCylinder( Vec3(0,0,0), Vec3(0,0,nerve_length), nerve_radius)
# SingleFascicle.Name = 'Epineurium'
# SingleFascicle.Transparency = 0.6
# TissueGroup.Add(SingleFascicle)

a_dia = [3,3.5,4,4.5,5,9.5,10,10.5,16]
weights = [20,20,20,20,20,15,15,15,10]

nerve_length, nerve_radius, centre_position = ref_epineurium()
axon_list = ref_fascicle_populator(5, a_dia, weights)

# cuff_position = 200
# cuff_length = 40
# BC_list = create_arraycuff(nerve_radius, cuff_position, 'Stimulation', centre_position, cuff_length, electrode_no, 2, False)

# cuff_position = 50
# electrode_no = 2
# BC_list2 = create_cuff(nerve_radius, cuff_position, 'Sensing', centre_position, cuff_length, electrode_no, True)

#axon_list = fascicle_creator(nerve_length,f_diameter,centre_position,axon_no,fascicle_no, a_dia, weights)

# tissue_width,_ = encapsulation_tissue(nerve_radius, centre_location, cuff_length, cuff_position, 'Stimulation', 0.6)

# cuff_list[0], stim_list, BC_list = create_cuff(tissue_width, cuff_position, 'Stimulation', centre_location, cuff_length, electrode_no, 2, False)
# cuff_position = 50
# cuff_list[1], sense_list, BC_list2 = create_cuff(nerve_radius, cuff_position, 'Sensing', centre_location, cuff_length, electrode_no, True)

#create_EM_sim(1000, f_list, peri_list, cuff_list[0], stim_list, pos_or_neg, BC_list, 'Stimulation')
#pos_or_neg=[1,-1]
#create_EM_sim(1000, f_list, peri_list, cuff_list[1], sense_list, pos_or_neg, BC_list2, 'Sensing')

#run_EM('Stimulation')
#run_EM('Sensing')

#create_NEURON_sim(10,0.0025, axon_list)
#run_NEURON('Nr')
#sim_name = 'Sensing'

# tissue_width = encapsulation_tissue(nerve_radius, centre_position, cuff_length, cuff_position, "Stimulation", 0.6)


['Model', 'Endoneurium_ 14', 'Endoneurium_ 31', 'Endoneurium_ 22', 'Endoneurium_ 35', 'Axon Trajectories', 'Endoneurium_ 5', 'Endoneurium_ 3', 'Endoneurium_ 33', 'Tissue', 'Endoneurium_ 16', 'Endoneurium_ 25', 'Endoneurium_ 13', 'Endoneurium_ 20', 'Endoneurium_ 30', 'Endoneurium_ 18', 'Endoneurium_ 24', 'Endoneurium_ 12', 'Cuffs', 'Endoneurium_ 34', 'Endoneurium_ 1', 'Endoneurium_ 6', 'Electrodes', 'Endoneurium_ 8', 'Endoneurium_ 27', 'Endoneurium_ 7', 'Contacts', 'Endoneurium_ 19', 'Perineurium', 'Endoneurium_ 17', 'Endoneurium_ 15', 'Endoneurium_ 11', 'Endoneurium', 'Boundary Conditions', 'Endoneurium_ 10', 'Endoneurium_ 4', 'Endoneurium_ 29', 'Endoneurium_ 32', 'Endoneurium_ 23', 'Grid', 'Endoneurium_ 36', 'Endoneurium_ 2', 'Endoneurium_ 9', 'Endoneurium_ 21', 'Endoneurium_ 26', 'Epineurium_Original', 'Endoneurium_ 28']
-2.4768 -1.20029 0  -1.02191 1.52555 300 
300.0 1.4549 2.72584


In [32]:
cuff_position = 200
cuff_length = 20
electrode_no = 6
BC_list = create_arraycuff(nerve_radius, cuff_position, 'Stimulation', centre_position, cuff_length, electrode_no, 2, False)

# cuff_position = 50
# electrode_no = 2
# BC_list2 = create_cuff(nerve_radius, cuff_position, 'Sensing', centre_position, cuff_length, electrode_no, True)

In [33]:
cuff_position = 50
electrode_no = 2
BC_list2 = create_cuff(nerve_radius, cuff_position, 'Sensing', centre_position, cuff_length, electrode_no, True)

In [34]:
pos_or_neg=[-1,0,0,0,0,0,2,2,2,2,2,2]
create_EM_sim(1000, pos_or_neg, BC_list, 'Stimulation')
pos_or_neg=[-1,1]
create_EM_sim(1000, pos_or_neg, BC_list2, 'Sensing')

In [35]:
create_NEURON_sim(20,0.0025, _)

In [44]:
nerve_length = 300
selectivity_sensor(36,nerve_length)

fascicle 0
157.25
fascicle 1
499.75
fascicle 2
499.75
fascicle 3
299.75
fascicle 4
428.25
fascicle 5
149.75
fascicle 6
428.25
fascicle 7
332.75
fascicle 8
428.25
fascicle 9
157.25
fascicle 10
157.25
fascicle 11
157.25
fascicle 12
149.75
fascicle 13
332.75
fascicle 14
332.75
fascicle 15
332.75
fascicle 16
157.25
fascicle 17
149.75
fascicle 18
499.75
fascicle 19
299.75
fascicle 20
428.25
fascicle 21
374.75
fascicle 22
428.25
fascicle 23
142.25
fascicle 24
93.25
fascicle 25
299.75
fascicle 26
299.75
fascicle 27
149.75
fascicle 28
332.75
fascicle 29
149.75
fascicle 30
142.25
fascicle 31
374.75
fascicle 32
428.25
fascicle 33
149.75
fascicle 34
374.75
fascicle 35
299.75
